# Import the list of csv files containing the measurements 

In [8]:
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join
import array
import struct
import csv  
from tqdm import tqdm

import librosa
import soundfile as sf

In [9]:
import soxr
import audioread
print(librosa.__version__)
print(soxr.__version__)
print(audioread.__version__)

0.10.2.post1
0.5.0.post1
3.0.1


In [10]:

mypath='F:/Data_BachelorHES/1.raw_measurements/'
onlyfiles = [join(mypath,f) for f in listdir(mypath) if isfile(join(mypath, f))]
print(onlyfiles)
del mypath

['F:/Data_BachelorHES/1.raw_measurements/PMPM_RAW_BIN_1m_20241116_085931.csv', 'F:/Data_BachelorHES/1.raw_measurements/PMPM_RAW_BIN_1m_20241116_090031.csv', 'F:/Data_BachelorHES/1.raw_measurements/PMPM_RAW_BIN_1m_20241116_090131.csv', 'F:/Data_BachelorHES/1.raw_measurements/PMPM_RAW_BIN_1m_20241116_090231.csv', 'F:/Data_BachelorHES/1.raw_measurements/PMPM_RAW_BIN_1m_20241116_090331.csv', 'F:/Data_BachelorHES/1.raw_measurements/PMPM_RAW_BIN_1m_20241116_090431.csv', 'F:/Data_BachelorHES/1.raw_measurements/PMPM_RAW_BIN_1m_20241116_090531.csv', 'F:/Data_BachelorHES/1.raw_measurements/PMPM_RAW_BIN_1m_20241116_090631.csv', 'F:/Data_BachelorHES/1.raw_measurements/PMPM_RAW_BIN_1m_20241116_090731.csv', 'F:/Data_BachelorHES/1.raw_measurements/PMPM_RAW_BIN_1m_20241116_090831.csv', 'F:/Data_BachelorHES/1.raw_measurements/PMPM_RAW_BIN_1m_20241116_090931.csv', 'F:/Data_BachelorHES/1.raw_measurements/PMPM_RAW_BIN_1m_20241116_091031.csv', 'F:/Data_BachelorHES/1.raw_measurements/PMPM_RAW_BIN_1m_2024111

In [11]:


filename_Training = np.array(onlyfiles)
print(filename_Training[-1])
del onlyfiles

F:/Data_BachelorHES/1.raw_measurements/PMPM_RAW_BIN_1m_20241116_114333.csv


## Import Raw Data to CSV for each sensor

In [ ]:

ind=0
destMeasurementsPath="F:/Data_BachelorHES/2.extracted_measurements/"

for file in filename_Training:
    values = []
    print('Reading file:',file)
    
    # Lire le fichier .csv qui contient le flux de bytes
    with open(file, 'rb') as f:
        byte_stream = f.read()  # Lire tout le fichier comme un flux de bytes
    num_values = len(byte_stream) // 2  # Calcul du nombre d'entiers (INT16)


    for i in range(num_values):
        two_bytes = byte_stream[i*2:(i*2)+2] #gather one byte and the next to be reconstructed as int16
        value = struct.unpack('<h', two_bytes)[0]  # '<h' pour 2 bytes en short int
        values.append(value) #values is a list of int16
        
    npValues=np.array(values, dtype=np.int16) #transform the list into a numpy array of np.int16 numbers
    if ind==0:
        sound_size=50 #size of the buffer of EL3632 (-1) Channel 1
        accelX_size=50#size of the buffer of EL3632 (-1) Channel 2
        accelY_size=50#size of the buffer of EL3632 (-2) Channel 1
        accelZ_size=50#size of the buffer of EL3632 (-2) Channel 2
        slice_size=sound_size+accelX_size+accelY_size+accelZ_size
        num_slice = len(npValues)//slice_size # Number of slices of 200 samples
        #Extract corresponding indices for each sensor for one file (will be repeated for each file) 
        sound_indices=np.concatenate([np.arange(i*slice_size, i*slice_size+sound_size) for i in range(num_slice)])
        accelX_indices=np.concatenate([np.arange(i*slice_size+sound_size, i*slice_size+sound_size+accelX_size) for i in range(num_slice)])
        accelY_indices=np.concatenate([np.arange(i*slice_size+sound_size+accelX_size, i*slice_size+sound_size+accelX_size+accelY_size) for i in range(num_slice)])
        accelZ_indices=np.concatenate([np.arange(i*slice_size+sound_size+accelX_size+accelY_size, i*slice_size+slice_size) for i in range(num_slice)])
        del sound_size, accelY_size, accelX_size, accelZ_size, slice_size, num_slice
    
    #Split each file into 4 csv files for each sensor    
    soundValues=npValues[sound_indices]
    accelXValues=npValues[accelX_indices]
    accelYValues=npValues[accelY_indices]
    accelZValues=npValues[accelZ_indices]
    
    #Send to a dictionnary -> pandasDataframe 
    myMeasures ={}
    myMeasures["Sound"]=soundValues.astype(np.int16, casting='safe')
    myMeasures["Accel X"]=accelXValues.astype(np.int16, casting='safe')
    myMeasures["Accel Y"]=accelYValues.astype(np.int16, casting='safe')
    myMeasures["Accel Z"]=accelZValues.astype(np.int16, casting='safe')
    myDfMeas=pd.DataFrame(myMeasures)
    
    #Write to csv file for each sensor. mode='a' indicate that we append to the file
    myDfMeas.to_csv(join(destMeasurementsPath,"sound.csv"), columns=["Sound"], index=False, mode='a', header=False)
    myDfMeas.to_csv(join(destMeasurementsPath,"accelX.csv"), columns=["Accel X"], index=False, mode='a', header=False)
    myDfMeas.to_csv(join(destMeasurementsPath,"accelY.csv"), columns=["Accel Y"], index=False, mode='a', header=False)
    myDfMeas.to_csv(join(destMeasurementsPath,"acceZX.csv"), columns=["Accel Z"], index=False, mode='a', header=False)
    ind+=1
    print("Data from file {} written to csv".format(file))
    del soundValues, accelXValues, accelYValues, accelZValues, myMeasures, myDfMeas, byte_stream, value, two_bytes, npValues
del num_values, values, sound_indices, accelX_indices, accelY_indices, accelZ_indices, ind, file, i
    
    


# Export sound data to a wavefile

In [12]:
sound = pd.read_csv('F:/Data_BachelorHES/2.extracted_measurements/sound.csv', header=None, names=['values'])
sound.head()
print(sound.shape)

#store the values from the pandas dataframe into a python array 
data_array = array.array('h', sound.values.flatten().astype(int))  # 'h' pour int16 (2 octets par valeur)
del sound

(495000000, 1)


In [13]:
print(np.max(data_array), np.min(data_array))

3098 -32768


In [14]:
# Convert python array into numpy array in float as to normalize the values between -1 and 1
data_array_np = np.array(data_array, dtype=np.float32)
print("Shape of numpy array:", data_array_np.shape)
print(f"Before normalization, Max value of numpy array: {np.max(data_array_np)} and Min value of numpy array: {np.min(data_array_np)}")
del data_array


Shape of numpy array: (495000000,)
Before normalization, Max value of numpy array: 3098.0 and Min value of numpy array: -32768.0


In [15]:
data_array_np = data_array_np/32768.0  # Normalization of values between -1 and 1
# We will resample the original sound data to 48kHz to match the audio bitrate from the video
fs_original = 50000  # 50 kHz
fs_target = 48000    # 48 kHz


In [18]:
# Resampling with librosa
resampled_data = librosa.resample(data_array_np, orig_sr=fs_original, target_sr=fs_target)
print(f"After normalization and resampling, Max value of numpy array:{np.max(resampled_data)} and Min value of numpy array: {np.min(resampled_data)}")
# resampling might affect original normalization, so we need to re-normalize

After normalization and resampling, Max value of numpy array:0.094422347843647 and Min value of numpy array: -1.0035430192947388


In [21]:
# Find max and min values
max_val = resampled_data.max()
min_val = resampled_data.min()
# Re-normalize between -1 and 1 and store into resampled_array
resampled_array = resampled_data / max(abs(max_val), abs(min_val))
print(f"After normalization of the resampled array, Max value of numpy array: {np.max(resampled_array)} and Min value of numpy array: {np.min(resampled_array)}")
del max_val, min_val

After normalization of the resampled array, Max value of numpy array: 0.09408898651599884 and Min value of numpy array: -1.0


In [22]:

#Export resampled array into a wav file with 16 bits PCM encoding for Audacity
sf.write('F:/Data_BachelorHES/4.audio_synchronization/resampled_audio_exp_sf.wav', resampled_array, fs_target, subtype='PCM_16')

In [23]:
# Export rsampled_array to another .wav file after amplification
gain_db = 20.0
amplification_factor = 10 ** (gain_db / 20)

# Apply amplification factor to the original data
amplified_signal = resampled_array * amplification_factor

# Normalize
amplified_signal = np.clip(amplified_signal, -1.0, 1.0)

#export
sf.write('F:/Data_BachelorHES/4.audio_synchronization/resampled_audio_exp_sf_amplified.wav', resampled_array, fs_target, subtype='PCM_16')

# Décalage trouvé avec Audacity
Situé le deuxième clap à environ 36 secondes après le début de la vidéo depuis la vidéo. Sur audacity, le pic du clap se trouve à l'échantillon 1'751'944 => @ 48KHz => 36.499s
Ce second clap est plus difficile à situer sur l'enregistrement du microphone. 
Pour réaliser une pré-synchronisation, on  utilisera une séquence de la machine particulièrement reconnaissable, une série de son "haute" fréquence audible à la fois dans l'enregistrement du microphone et dans l'enregistrement de la vidéo. Pour situer ces sons haute fréquences, on visualise les stream audio en spectrogramme dans Audacity. On regarde à quel échantillon approximatif ils commencent dans les deux streams. On calcule le décalage entre les deux streams et on l'applique à l'enregistrement de la vidéo pour synchroniser les deux streams. Cette pré-synchronisation nous donne un décalage de 14'874'177 échantillons (soit 5min 9s 879ms). A partir de là, on sait qu' à ~10'000 échantillons prés (~200ms), nos flux sont synchronisés. 
Après ce décalage, le second clap se trouve à l'échantillon 16'626'121 sur le son de la vidéo.
On repasse ensuite sur la zone du second clap pour tenter d'entendre ce clap sur le son du microphone. Et heureusement on l'entend assez distinctement. Cependant le pic n'est pas aussi marqué que sur l'enregistrement du son de la vidéo. Il s'agit plutôt d'une vague aplatie avec un dizaine d'échantillons de largeur. On peut donc considérer que le clap se situe entre les échantillons 16'627'385 et 16'627'394. On peut donc considérer que le clap se situe au centre de cette vague, soit à l'échantillon 16'627'390. Ainsi le clap du microphone à 16'627'390 et celui de la vidéo à 16'626'121 sont décalés de 1'269 échantillons (soit 26ms).  Le décalage total se porte ainsi à 14'875'446 échantillons (soit 5min 9s 905ms).

Il est intéressant de noter que le pic du clap est aussi décalé entre les deux flux "stéréo" de la vidéo d'environ 7 échantillons. Ce décalage est dû à plusieurs paramêtres: placement des deux micro du téléphone, distance entre le clap et le téléphone, traitement interne du signal par le téléphone (ADC, DSP etc..) etc..  Ainsi on peut déterminer que notre synchronisation est correct avec une tolérance de +/- 100 échantillons. Car le micro de la machine et de la caméra ne sont pas situés au même endroit et que le clap n'est pas un son ponctuel mais un son d'une certaine durée.  

Pour la suite de l'analyse nous jetterons ainsi les échantillons 0 à 14'875'446 de l'enregistrement du son du microphone.
Cependant comme nous avons travaillé avec Audacity sur un projet avec un tx d'échantillonnage à 48kHz, alors que nous avons des valeurs sur 50kHz (14'875'446*50/48), nous devons exclure les échantillons 0 à 15'495'256 pour le microphone et les 3 axes de l'accéléromètre.


Ensuite, lors de l'OCR nous avons dû exclure les frame 0 à 9964 (soit les 9965 premières frames) et que nous avons un temps en millisecondes jusqu'à la frame rate 9965 de 332'245.6 ms, nous devons également exclure les 15'947'789 échantillons suivants (à 48kHz) de l'enregistrement du microphone et du son de la vidéo. Ainsi il faut exclure les enregistrements 0 à 30'823'235 pour le microphone et les 3 axes de l'accéléromètre, ce qui ramené à la fréquence d'échantillonnage originale de 50kHz, correspond à l'échantillon 32'107'536.

 

## Map the frames to the corresponding samples for microphone and accelerometer

In [2]:
class FrameInfo:
    def __init__(self, frame_number, time_in_milliseconds, measCount, measLowIndex, HighIndex):
        self.frame_number = frame_number
        self.time_in_milliseconds = time_in_milliseconds
        self.measCount=measCount
        self.measLowIndex=measLowIndex
        self.HighIndex=HighIndex
    
    def __repr__(self):
        return f"Frame {self.frame_number} at {self.time_in_milliseconds} ms with {self.measCount} measurements at index {self.measLowIndex} to {self.HighIndex}"
    
    def getNbMeas(self):
        return self.measCount
    def getMeasIndex(self):
        return (self.measLowIndex, self.HighIndex)

In [3]:
import cv2
video_path = 'F:/Data_BachelorHES/3.Video_CNC/Video_CNC.mp4'
cap = cv2.VideoCapture(video_path)

In [4]:

frameData=[]

count=9965
cap.set(cv2.CAP_PROP_POS_FRAMES, count)
cap.read()
while cap.isOpened():
        
    actualTime=cap.get(cv2.CAP_PROP_POS_MSEC)
    cap.set(cv2.CAP_PROP_POS_FRAMES, count+5)
    
    ret, frame = cap.read()
    if not ret:
        frameData.append(FrameInfo(count, actualTime, 0, highIndex, highIndex))
        print("End of stream ... Exiting ...")
        break
    
   
    nextTime=cap.get(cv2.CAP_PROP_POS_MSEC)
    lowIndex=15495256 + int(actualTime*50)
    highIndex=15495256 + int(nextTime*50)
    totMeas=highIndex-lowIndex
    frameData.append(FrameInfo(count, actualTime, totMeas, lowIndex, highIndex))
    count+=5
del totMeas, ret, highIndex, lowIndex, frame, count, actualTime, nextTime


End of stream ... Exiting ...


In [6]:
frameData[-1].frame_number

Frame 285545 at 9520340.066666668 ms with 0 measurements at index 491512259 to 491512259

In [11]:
print(frameData[-1])

Frame 285545 at 9520340.066666668 ms with 0 measurements at index 491512259 to 491512259


In [18]:
#Add infos for the last frame which last 1 frame (that is 1/fps)
fps = cap.get(cv2.CAP_PROP_FPS)
duration=1.0/fps
totMeasurements=int(duration*50000)
lowIndex=15495256 + int(frameData[-1].time_in_milliseconds*50)
highIndex=lowIndex+totMeasurements
frameData[-1].measCount=totMeasurements
frameData[-1].measLowIndex=lowIndex
frameData[-1].HighIndex=highIndex
print(frameData[-1])
del highIndex, lowIndex, totMeasurements, duration, fps

Frame 285545 at 9520340.066666668 ms with 1667 measurements at index 491512259 to 491513926
